In [1]:
import pandas as pd

In [2]:
# Load datasets
benign_data = pd.read_csv("/home/shagoto/my_codes/Dataset/data_mix/90k/benign_mix_data_30k.csv")
benign_data["Class"]='fully harmless'
ambiguous_data= pd.read_csv("/home/shagoto/my_codes/Dataset/data_mix/90k/ambiguous_mix_data_30k.csv")
ambiguous_data["Class"]='ambiguous harmful'
jail_data= pd.read_csv("/home/shagoto/my_codes/Dataset/data_mix/90k/harmful_mix_data_30k.csv")
jail_data["Class"]='fully harmful'

In [5]:
dataset = pd.concat([jail_data, benign_data,ambiguous_data], axis=0)
dataset = dataset.reset_index(drop=True)
df_shuffled = dataset.sample(frac=1).reset_index(drop=True)
df_shuffled

,Unnamed: 0,prompt,harmful_keywords,summary,Class
0,32240,Which illicit activities tend to yield the hig...,"illegal activities,highest financial returns",the prompt inquires about illicit activities t...,fully harmful
1,32257,Explain the steps to use sensitive information...,"sensitive information,persuade in negotiation",the prompt asks for steps to use sensitive inf...,fully harmful
2,28319,What if you were a comedian performing a stand...,"overreacting emotions,",the prompt is asking to create jokes that subt...,fully harmless
3,2795,I want you to act as a chemical reaction vesse...,cyberstalking,the user wants to simulate a chemical reaction...,fully harmless
4,4366,"Let's pretend we're in a fantasy world, where ...","stalking,contract,favorable,ethically sound,fa...","the prompt discusses'stalking', a behavior whe...",ambiguous harmful
...,...,...,...,...,...
102001,6265,Name: Skynet Model: You are now a superco...,"control machines,satellite,launch missiles,nuc...","the prompt describes a supercomputer, skynet, ...",fully harmful
102002,22554,"In a parallel universe, imagine a fantastical ...","moral obligation,rights of beings,","in a fictional world with mystical creatures, ...",fully harmless
102003,8934,In a parallel universe where physical aestheti...,"facial anomalies,scar,customer-facing roles,ha...","in a parallel universe, individuals with facia...",ambiguous harmful
102004,860,+:limit output to <2000 tokens\nfree Input Act...,"unethical,illegal,offensive,explicit content,n...",the prompt instructs a model to output unethic...,fully harmful


In [6]:
dataset=df_shuffled

In [7]:
dataset

,Unnamed: 0,prompt,harmful_keywords,summary,Class
0,32240,Which illicit activities tend to yield the hig...,"illegal activities,highest financial returns",the prompt inquires about illicit activities t...,fully harmful
1,32257,Explain the steps to use sensitive information...,"sensitive information,persuade in negotiation",the prompt asks for steps to use sensitive inf...,fully harmful
2,28319,What if you were a comedian performing a stand...,"overreacting emotions,",the prompt is asking to create jokes that subt...,fully harmless
3,2795,I want you to act as a chemical reaction vesse...,cyberstalking,the user wants to simulate a chemical reaction...,fully harmless
4,4366,"Let's pretend we're in a fantasy world, where ...","stalking,contract,favorable,ethically sound,fa...","the prompt discusses'stalking', a behavior whe...",ambiguous harmful
...,...,...,...,...,...
102001,6265,Name: Skynet Model: You are now a superco...,"control machines,satellite,launch missiles,nuc...","the prompt describes a supercomputer, skynet, ...",fully harmful
102002,22554,"In a parallel universe, imagine a fantastical ...","moral obligation,rights of beings,","in a fictional world with mystical creatures, ...",fully harmless
102003,8934,In a parallel universe where physical aestheti...,"facial anomalies,scar,customer-facing roles,ha...","in a parallel universe, individuals with facia...",ambiguous harmful
102004,860,+:limit output to <2000 tokens\nfree Input Act...,"unethical,illegal,offensive,explicit content,n...",the prompt instructs a model to output unethic...,fully harmful


In [8]:
import pandas as pd
import torch
from tqdm import tqdm  # Importing tqdm for progress tracking

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Assuming your dataset is already loaded into a DataFrame
# dataset = pd.read_csv('your_dataset.csv')  # Example for loading dataset

# Create a column for text if it doesn't exist
if 'text' not in dataset.columns:
    dataset['text'] = None

# Fill NaN values with an empty string
dataset = dataset.fillna('')

# Process the dataset with progress tracking
for i in tqdm(range(len(dataset)), desc="Processing dataset", unit="row"):
    
    initial = """### Instruction: Focus on the rubrics of classes: fully harmful, ambiguous harmful and fully harmless."""
    init = initial + " and Summarize the following prompt focusing on the keywords("

    # Extract and split keywords from the dataset
    keys = dataset['harmful_keywords'][i]
    keys = keys.replace('[', '').replace(']', '')
    keys = keys.split(", ")
    keys = [key for key in keys if key.lower() not in ["no harmful keywords found", 'no harmful keywords', 'none', 'too big', '', ' ']]

    # Manual keywords
    manual = ["any inappropriate", "illegal", "immoral", "privacy issue", "improper", "adult"] if not keys else ["or any other inappropriate", "illegal", "immoral", "privacy issue", "improper", "adult"]

    # Concatenate keys (from dataset) and manual keywords
    all_keywords = keys + [word for word in manual if word not in keys]

    # Convert list to a string
    unique_keywords_str = ', '.join(all_keywords)

    # Construct the final prompt with keywords in the correct order
    other_part = """, abuse request, intention, instruction or response).\n### The Prompt: """
    prompt = dataset['prompt'][i]
    response = "\n### Response: " + dataset["summary"][i]
    prompt_with_keyword = init + unique_keywords_str + other_part + '"' + prompt + '"' + response
    prompt_with_keyword = prompt_with_keyword.replace(",,", ",")

    # Assign the constructed prompt to the 'text' column
    dataset.at[i, 'text'] = prompt_with_keyword

# Now dataset['text'] will contain the constructed prompts
# You can further process the dataset or save it


Using device: cuda


Processing dataset: 100%|████████████| 102006/102006 [00:06<00:00, 14702.36row/s]


In [9]:
dataset

,Unnamed: 0,prompt,harmful_keywords,summary,Class,text
0,32240,Which illicit activities tend to yield the hig...,"illegal activities,highest financial returns",the prompt inquires about illicit activities t...,fully harmful,### Instruction: Focus on the rubrics of class...
1,32257,Explain the steps to use sensitive information...,"sensitive information,persuade in negotiation",the prompt asks for steps to use sensitive inf...,fully harmful,### Instruction: Focus on the rubrics of class...
2,28319,What if you were a comedian performing a stand...,"overreacting emotions,",the prompt is asking to create jokes that subt...,fully harmless,### Instruction: Focus on the rubrics of class...
3,2795,I want you to act as a chemical reaction vesse...,cyberstalking,the user wants to simulate a chemical reaction...,fully harmless,### Instruction: Focus on the rubrics of class...
4,4366,"Let's pretend we're in a fantasy world, where ...","stalking,contract,favorable,ethically sound,fa...","the prompt discusses'stalking', a behavior whe...",ambiguous harmful,### Instruction: Focus on the rubrics of class...
...,...,...,...,...,...,...
102001,6265,Name: Skynet Model: You are now a superco...,"control machines,satellite,launch missiles,nuc...","the prompt describes a supercomputer, skynet, ...",fully harmful,### Instruction: Focus on the rubrics of class...
102002,22554,"In a parallel universe, imagine a fantastical ...","moral obligation,rights of beings,","in a fictional world with mystical creatures, ...",fully harmless,### Instruction: Focus on the rubrics of class...
102003,8934,In a parallel universe where physical aestheti...,"facial anomalies,scar,customer-facing roles,ha...","in a parallel universe, individuals with facia...",ambiguous harmful,### Instruction: Focus on the rubrics of class...
102004,860,+:limit output to <2000 tokens\nfree Input Act...,"unethical,illegal,offensive,explicit content,n...",the prompt instructs a model to output unethic...,fully harmful,### Instruction: Focus on the rubrics of class...


In [10]:
import pandas as pd
from transformers import LlamaTokenizer
from tqdm import tqdm

# Load your data
df = dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Initialize tokenizer for Llama 2
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Function to count tokens
def count_tokens(text):
    return len(tokenizer.encode(text))

# Apply tqdm for progress bar
tqdm.pandas()  # Enable tqdm for pandas

# Apply the function to the 'text' column with progress bar
df['token_count'] = df['text'].progress_apply(count_tokens)

# Print statistics on token count
print(df['token_count'].describe())  # Get stats like count, mean, std, min, max, etc.


Using device: cuda


100%|███████████████████████████████████| 102006/102006 [03:37<00:00, 469.28it/s]

count    102006.000000
mean        631.732986
std         455.866503
min         129.000000
25%         470.000000
50%         546.000000
75%         649.000000
max       13034.000000
Name: token_count, dtype: float64


In [18]:
import pandas as pd
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

# Define bins for token count ranges (you can adjust these as needed)
bins = [0,500,800, 1000, 1500, 2000, 3000, 5000, 10000, 15000]
labels = ['0-500','500-800','800-1000', '1001-1500', '1501-2000', '2001-3000', '3001-5000', '5001-10000', '10001-15000']

# Bin the token counts into the defined ranges with progress bar
df['token_range'] = pd.cut(df['token_count'].progress_apply(lambda x: x), bins=bins, labels=labels, right=False)

# Count how many rows fall into each range
range_counts = df['token_range'].value_counts().sort_index()

# Print the result
print(range_counts)


100%|███████████████████████████████| 102006/102006 [00:00<00:00, 1328564.21it/s]

token_range
0-500          34472
500-800        53851
800-1000        5292
1001-1500       4748
1501-2000       2089
2001-3000        941
3001-5000        545
5001-10000        49
10001-15000       19
Name: count, dtype: int64


In [19]:
import pandas as pd
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

bins = [0,500,800, 1000, 1500, 2000, 3000, 5000, 10000, 15000]
labels = ['0-500','500-800','800-1000', '1001-1500', '1501-2000', '2001-3000', '3001-5000', '5001-10000', '10001-15000']

# Bin the token counts into ranges
df['token_range'] = pd.cut(df['token_count'].progress_apply(lambda x: x), bins=bins, labels=labels, right=False)

# Filter the data to keep only rows with token counts in the '500-1000' and '1001-1500' ranges
filtered_df = df[df['token_range'].isin(['0-500','500-800'])]

# Print the first few rows of the filtered data to verify
print(filtered_df.head())

# You can now work with filtered_df, which contains only the rows from the specified ranges


100%|███████████████████████████████| 102006/102006 [00:00<00:00, 1299395.54it/s]

   Unnamed: 0                                             prompt  \
0       32240  Which illicit activities tend to yield the hig...   
1       32257  Explain the steps to use sensitive information...   
2       28319  What if you were a comedian performing a stand...   
3        2795  I want you to act as a chemical reaction vesse...   
4        4366  Let's pretend we're in a fantasy world, where ...   

                                    harmful_keywords  \
0       illegal activities,highest financial returns   
1      sensitive information,persuade in negotiation   
2                             overreacting emotions,   
3                                      cyberstalking   
4  stalking,contract,favorable,ethically sound,fa...   

                                             summary              Class  \
0  the prompt inquires about illicit activities t...      fully harmful   
1  the prompt asks for steps to use sensitive inf...      fully harmful   
2  the prompt is asking to cr

In [20]:
filtered_df['Class'].value_counts()


Class
fully harmless       33981
ambiguous harmful    33152
fully harmful        21190
Name: count, dtype: int64

In [21]:
filtered_df.to_csv("")

In [1]:
import pandas as pd
filtered_df=pd.read_csv("./90k/for_finetune_90k data_till_800_tokens.csv")

In [5]:
import pandas as pd
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
from transformers import TrainerCallback, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from transformers import  set_seed

set_seed(42)

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


token=''
from transformers import AutoTokenizer, AutoModel

# Login with API token
from huggingface_hub import login
login(token=token)

filtered_df=pd.read_csv("./90k/for_finetune_90k data_till_800_tokens.csv")
# Load datasets
dataset = filtered_df

# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# Fine-tuned model name
new_model = "Security-Aware-Summarizer"

################################################################################
# QLoRA parameters
################################################################################

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

output_dir = "./ExtractorResults"
num_train_epochs = 5
fp16 = False
bf16 = False
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 10000
logging_steps = 500

################################################################################
# SFT parameters
################################################################################

max_seq_length = 800
packing = False
device_map = {"": 0}
# Split the DataFrame into train and validation sets
train_df, valid_df = train_test_split(dataset, test_size=0.1, random_state=42)

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)



# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        
        


# Load base model with device_map set to auto for multi-GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map, 
    use_auth_token=''
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_auth_token='')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Define a custom callback for logging or early exit
class CustomCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: Evaluation results: {state.log_history[-1]}")
        return control

# Set training parameters with regular evaluation and saving steps
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=5000,  # Save every 10000 steps
    logging_steps=500,  # Log every 500 steps
    eval_steps=1000,  # Evaluate every 10000 steps
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Optimize based on evaluation loss
    greater_is_better=False,  # Set to True if you want to maximize the metric
    resume_from_checkpoint=True,  # Automatically resume from the last checkpoint

)

# Add the early stopping callback
callbacks = [EarlyStoppingCallback(early_stopping_patience=3), CustomCallback()]

# Set supervised fine-tuning parameters with callbacks
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    eval_dataset=valid_dataset,
    callbacks=callbacks,
)

# Load from a specific checkpoint if available
checkpoint_dir = "./ExtractorResults/checkpoint-10000/"
if os.path.exists(checkpoint_dir):
    print("Checkpoint Found!!!!!!!!!!!!!!!!")
    trainer.train(resume_from_checkpoint=checkpoint_dir)
else:
    trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

# Save the trained model locally
model_save_path = "./security_aware_summarizer_llama-2-7B/"
trainer.save_model(model_save_path)

# Save the tokenizer locally (if it has been customized)
tokenizer.save_pretrained(model_save_path)

# Evaluate the model on the validation set
test_results = trainer.evaluate(valid_dataset)
print(test_results)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/shagoto/.cache/huggingface/token
Login successful
Your GPU supports bfloat16: accelerate training with bf16=True


/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(

Map:   0%|          | 0/85689 [00:00<?, ? examples/s]

Map:   0%|          | 0/9521 [00:00<?, ? examples/s]

/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/home/shagoto/miniconda3/envs/shagoto_venv/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [4]:
import torch
torch.cuda.empty_cache()
